# 结果集

In [ ]:
import numpy as np
import pandas as pd

from QSExt.GUI.Notebook import showOutput
Bar2 = pd.DataFrame(np.random.randn(3,2), index=["中文", "b2", "b3"], columns=["中文", "我是个例子"])
Bar2.iloc[0,0] = np.nan
TestData = {"Bar1":{"a": {"a1": pd.DataFrame(np.random.rand(50,3)),
                          "a2": pd.DataFrame(np.random.rand(4,3))},
                    "b": pd.DataFrame(['a']*150,columns=['c'])},
                     "Bar2": Bar2}
Dlg = showOutput(data=TestData)

In [ ]:
Dlg.Widgets["GetArgsDlg"]["ArgSetupDlg"].Args

# 时点设置

In [ ]:
import datetime as dt

import QuantStudio.api as QS
from QSExt.FactorDataBase.AKShareDB import AKShareDB
from QSExt.GUI.Notebook.DateTimeSetupDlg import DateTimeSetupDlg

AKSDB = AKShareDB().connect()
HDB = QS.FactorDB.HDF5DB().connect()

DTs = [dt.datetime(2022, 1, 1), dt.datetime(2022, 1, 2)]
FTs = [HDB.getTable("stock_cn_day_bar")]
Dlg = DateTimeSetupDlg(dts=DTs, trading_day_fun=AKSDB.getTradeDay, fts=FTs)
Dlg.display()

# ID 设置

In [ ]:
import datetime as dt

import QuantStudio.api as QS
from QSExt.FactorDataBase.AKShareDB import AKShareDB
from QSExt.GUI.Notebook.IDSetupDlg import IDSetupDlg

AKSDB = AKShareDB().connect()
HDB = QS.FactorDB.HDF5DB().connect()

IDs = ["000001.SZ", "000002.SZ"]
FTs = [HDB.getTable("stock_cn_day_bar")]
# IDFun = {"A股": lambda date, is_current: AKSDB.getStockID(date=None, is_current=False)}
IDFun = {"测试": lambda date, is_current: ["000002.SZ", "000003.SZ"]}
Dlg = IDSetupDlg(ids=IDs, id_fun=IDFun, fts=FTs)
Dlg.display()

# 因子图

In [3]:
import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools
from QSExt.GUI.Notebook.FactorGraphDlg import FactorGraphDlg

HDB = QS.FactorDB.HDF5DB().connect()

# Factor1 = QS.FactorDB.DataFactor(name="Factor1", data=1)
# Factor2 = QS.FactorDB.DataFactor(name="Factor2", data=2)
FT = HDB.getTable("stock_cn_test")
Factor1 = FT.getFactor("Factor3", new_name="因子1")
Factor2 = FT.getFactor("Factor4", new_name="因子2")
Factor3 = fd.rolling_cov(Factor1, Factor2, window=5, factor_name="因子3")
Factor4 = QS.FactorDB.Factorize((Factor1 + Factor2 * Factor3 ** 2) / 2, factor_name="因子4")
Factor5 = fd.nansum(Factor1 & Factor2, Factor4, factor_name="因子5")

CFT = QS.FactorDB.CustomFT(name="MyFT")
CFT.addFactors(factor_list=[Factor2, Factor5])

Dlg = FactorGraphDlg(factor_list=[Factor1, Factor2, Factor3, Factor4, Factor5], cfts=[CFT])
Dlg.display()